In [1]:
import pandas as pd
import numpy as np

In [4]:
olist_orders = pd.read_csv("../data/raw/archive/olist_orders_dataset.csv")
olist_order_items = pd.read_csv("../data/raw/archive/olist_order_items_dataset.csv")
olist_customers = pd.read_csv("../data/raw/archive/olist_customers_dataset.csv")

# Processamento
- Junção das Tabelas
- Filtro de Variáveis
- Construção de features:
    - Faturamento Total por Cliente
    - Quantidade Total de Pedidos por Cliente

In [7]:
# Junção de tabelas
customers_orders = olist_customers.merge(olist_orders, how="inner", on="customer_id")
customers_orders_items = customers_orders.merge(olist_order_items, how="inner", on="order_id")

In [9]:
customers_orders_items.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'order_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

In [10]:
#Filtro de Variáveis
customers_orders_items.drop(columns=[
                                        i for i in customers_orders_items.columns if i not in ['customer_unique_id', 'order_id', 'order_status', 'price', 'freight_value']
                                    ], 
                            inplace = True)

In [11]:
customers_orders_items

,customer_unique_id,order_id,order_status,price,freight_value
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,delivered,124.99,21.88
1,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,delivered,289.00,46.48
2,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,139.94,17.79
3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,delivered,149.94,23.36
4,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,230.00,22.25
...,...,...,...,...,...
112645,1a29b476fee25c95fbafc67c5ac95cf8,6760e20addcf0121e9d58f2f1ff14298,delivered,74.90,13.88
112646,d52a67c98be1cf6a5c84435bd38d095d,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,114.90,14.16
112647,e9f50caf99f032f0bf3c55141f019d99,fed4434add09a6f332ea398efd656a5c,delivered,37.00,19.04
112648,73c2643a0a458b49f58cea58833b192e,e31ec91cea1ecf97797787471f98a8c2,delivered,689.00,22.07


In [12]:
customers_orders_items.order_status.unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'canceled',
       'unavailable', 'approved'], dtype=object)

In [16]:
customers_orders_items = customers_orders_items[(customers_orders_items.order_status != 'unavailable') & (customers_orders_items.order_status != 'canceled')]

In [17]:
customers_orders_items

,customer_unique_id,order_id,order_status,price,freight_value
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,delivered,124.99,21.88
1,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,delivered,289.00,46.48
2,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,139.94,17.79
3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,delivered,149.94,23.36
4,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,230.00,22.25
...,...,...,...,...,...
112645,1a29b476fee25c95fbafc67c5ac95cf8,6760e20addcf0121e9d58f2f1ff14298,delivered,74.90,13.88
112646,d52a67c98be1cf6a5c84435bd38d095d,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,114.90,14.16
112647,e9f50caf99f032f0bf3c55141f019d99,fed4434add09a6f332ea398efd656a5c,delivered,37.00,19.04
112648,73c2643a0a458b49f58cea58833b192e,e31ec91cea1ecf97797787471f98a8c2,delivered,689.00,22.07


In [22]:
values_by_customer = customers_orders_items.groupby(by="customer_unique_id").sum()

In [20]:
olist_customers.customer_unique_id.unique().shape

(96096,)

In [24]:
values_by_customer["total_value"] = values_by_customer["price"] + values_by_customer["freight_value"]

In [26]:
values_by_customer.describe()

,price,freight_value,total_value
count,94983.000000,94983.000000,94983.000000
mean,142.071747,23.595025,165.666772
std,216.074999,22.867488,226.726478
min,0.850000,0.000000,9.590000
25%,47.900000,14.090000,63.100000
50%,89.890000,17.600000,107.900000
75%,155.000000,25.540000,182.930000
max,13440.000000,1794.960000,13664.080000


In [31]:
customers_orders = customers_orders[(customers_orders.order_status != 'unavailable') & (customers_orders.order_status != 'canceled') & (customers_orders.order_status != 'created')]

In [40]:
quant_orders_by_cust = customers_orders.groupby(by="customer_unique_id").count()["order_id"]
quant_orders_by_cust = quant_orders_by_cust.reset_index(level=0)

In [41]:
quant_orders_by_cust

,customer_unique_id,order_id
0,0000366f3b9a7992bf8c76cfdf3221e2,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1
2,0000f46a3911fa3c0805444483337064,1
3,0000f6ccb0745a6a4b88665a16c9f078,1
4,0004aac84e0df4da2b147fca70cf8255,1
...,...,...
94981,fffcf5a5ff07b0908bd4e2dbc735a684,1
94982,fffea47cd6d3cc0a88bd621562a9d061,1
94983,ffff371b4d645b6ecea244b27531430a,1
94984,ffff5962728ec6157033ef9805bacc48,1


In [43]:
values_by_customer = values_by_customer.reset_index(level=0)

In [46]:
customers_info = values_by_customer.merge(quant_orders_by_cust, how="inner", on="customer_unique_id")

In [47]:
customers_info.drop(columns=["price", "freight_value"], inplace=True)
customers_info.rename(columns={"order_id": "quant_orders"}, inplace=True)

In [49]:
customers_info.describe()

,total_value,quant_orders
count,94983.000000,94983.000000
mean,165.666772,1.033859
std,226.726478,0.210811
min,9.590000,1.000000
25%,63.100000,1.000000
50%,107.900000,1.000000
75%,182.930000,1.000000
max,13664.080000,16.000000


In [50]:
customers_info.to_csv("../data/processed/customers_info.csv")